In [2]:
print("Starting To Printing")
counter = 0
for i in range(10):
    counter = i
    print("The count is %d" % counter)
    print("-------------------------")
print("Finished Printing")

Starting To Printing
The count is 0
-------------------------
The count is 1
-------------------------
The count is 2
-------------------------
The count is 3
-------------------------
The count is 4
-------------------------
The count is 5
-------------------------
The count is 6
-------------------------
The count is 7
-------------------------
The count is 8
-------------------------
The count is 9
-------------------------
Finished Printing


In [11]:
import urllib.request
sites = [
    'https://www.yahoo.com',
    'http://www.cnn.com',
    'http://www.python.org',
    'http://www.jython.org',
    'http://www.perl.org',
    'http://www.cisco.com',
    'http://www.facebook.com',
    'http://www.twitter.com',
    'http://www.github.com',
    'http://www.linkedin.com',
]

for url in sites:
    with urllib.request.urlopen(url) as u:
        page = u.read()
        print(url, len(page))


https://www.yahoo.com 484557
http://www.cnn.com 1570281
http://www.python.org 49097
http://www.jython.org 19210
http://www.perl.org 12630
http://www.cisco.com 86651
http://www.facebook.com 519103
http://www.twitter.com 172676
http://www.github.com 65753
http://www.linkedin.com 48162


In [12]:
counter = 0


def worker():
    "My Job is to increment the counter and print the current function."
    global counter
    counter += 1
    print("The counter is %d" % counter)
    print("---------------------------")

    
print("Starting Up")

for i in range(10):
    worker()
    
print("Finishing Up")

Starting Up
The counter is 1
---------------------------
The counter is 2
---------------------------
The counter is 3
---------------------------
The counter is 4
---------------------------
The counter is 5
---------------------------
The counter is 6
---------------------------
The counter is 7
---------------------------
The counter is 8
---------------------------
The counter is 9
---------------------------
The counter is 10
---------------------------
Finishing Up


In [15]:
import threading
counter = 0


def worker():
    "My Job is to increment the counter and print the current function."
    global counter
    counter += 1
    print("The counter is %d" % counter)
    print("---------------------------")

    
print("Starting Up")

for i in range(10):
    threading.Thread(target=worker).start()
    
print("Finishing Up")

Starting Up
The counter is 1
---------------------------
The counter is 2
---------------------------
The counter is 3The counter is 4
---------------------------
The counter is 5

---------------------------
---------------------------
The counter is 6The counter is 7
---------------------------

---------------------------
The counter is 8
---------------------------
The counter is 9
---------------------------
The counter is 10
---------------------------
Finishing Up


### Race Conditions - Fuzzing

In [17]:
import threading, time, random


################################
# Fuzzing is a technique for amplifying race condition errors to make them more visible.
FUZZ = True
def fuzz():
    if FUZZ:
        time.sleep(random.random())
#################################


counter = 0


def worker():
    "My Job is to increment the counter and print the current function."
    global counter
    fuzz()
    oldcnt = counter
    fuzz()
    counter = oldcnt + 1
    fuzz()
    print("The counter is %d" % counter, end=' ')
    fuzz()
    print()
    fuzz()
    print("---------------------------", end=' ')
    fuzz()
    print 
    fuzz()
    
print("Starting Up")
fuzz()

for i in range(10):
    threading.Thread(target=worker).start()
    fuzz()
    
print("Finishing Up")
fuzz()

Starting Up
The counter is 1 
The counter is 2 --------------------------- The counter is 2 
The counter is 3 

--------------------------- --------------------------- The counter is 4 


The counter is 4 

The counter is 4 Finishing Up
--------------------------- --------------------------- --------------------------- 

The counter is 6 The counter is 6 
The counter is 6 --------------------------- 


--------------------------- 
--------------------------- 


--------------------------- 


In [ ]:
import queue
import threading

counter = 0

counter_queue = queue.Queue()
def counter_manager():
    'I have EXCLUSIVE rights to update the counter variable'
    global counter
    
    while True:
        increment = counter_queue.get()
        counter += increment
        print_queue.put(['The count is %d' % counter,
                         '---------------------------'])
        counter_queue.task_done()

t = threading.Thread(target=counter_manager)
t.daemon = True
t.start()
del t

    
print_queue = queue.Queue()
def print_manager():
    "I have exclusive right to call the print keyworkd"
    while True:
        job = print_queue.get()
        for line in job:
            print(line)
        print_queue.task_done()


t = threading.Thread(target=print_manager)
t.daemon = True
t.start()
del t


def worker():
    "My hob is to increament the counter and print the current count."
    counter_queue.put(1)

print_queue.put(['Starting Up'])
worker_threads = []

for i in range(10):
    t = threading.Thread(target=worker)
    worker_threads.append(t)
    t.start()

for t in worker_threads:
    t.join()
counter_queue.join()
print_queue.put(['Finishing Up'])
print_queue.join()